# **CH01.2. GloVe**

#### **`Paper Info`** : GloVe: Global Vectors for Word Representation
#### $ \hspace{1.75cm} - $ Jeffrey Pennington, Richard Socher, Christopher D. Manning et al. (EMNLP/2014)

<b></b>

> ## **요약(Summary)**

| Item | Description |
|------|-------------|
| Research topic | 전역 word-word co-occurrence 통계를 log-bilinear 회귀로 근사하여 단어 임베딩을 학습하는 GloVe 모델 제안 |
| Key findings | $ \cdot{} $ 단어 의미 구분은 probe 단어 $ k $에 대한 조건부 동시출현 확률의 비율 $ P(k|i)/P(k|j) $로 설명 가능하며, 해당 비율의 로그가 임베딩 내적과 선형적으로 대응함 <br> $ \cdot{} $ weighted least squares 목적함수와 weighting function $ f(x) $ 설계로 희귀·초빈 단어의 영향을 균형화함 <br> $ \cdot{} $ Word analogy, similarity, NER 등에서 Word2Vec 대비 경쟁력 있는 성능을 보임 |
| Contributions | $ \cdot{} $ co-occurrence 기반 통계적 정보와 예측 기반 모델의 장점을 결합함 <br> $ \cdot{} $ 로그-회귀 기반의 단순하면서도 해석 가능한 학습 구조를 제시함 <br> $ \cdot{} $ 대규모 코퍼스에서도 효율적으로 학습 가능한 확장성을 입증함 |

<b></b>

> ## **연구 배경(Motivation & Prior Work)**

#### **(1) 기존 연구 및 한계점** :
##### $ \hspace{0.15cm} \cdot{} $ LSA 등 count-based 방법은 전역 통계를 활용하지만 단어 간 선형 의미 관계를 반영하지 못함
##### $ \hspace{0.15cm} \cdot{} $ Skip-gram, CBOW 등 predict-based 방법은 의미 관계를 잘 학습하지만 전역 통계 정보 활용이 제한적임
##### $ \hspace{0.15cm} \cdot{} $ 두 접근의 장점을 통합한 이론적·실용적 학습 틀이 부재함

#### **(2) 연구 목표** :
##### $ \hspace{0.15cm} \cdot{} $ 전역 통계 활용 효율성과 예측 기반 선형 의미 구조를 동시에 달성
##### $ \hspace{0.15cm} \cdot{} $ 단어 의미 차이를 조건부 동시출현 확률의 비율로 모델링
##### $ \hspace{0.15cm} \cdot{} $ 단순하고 해석 가능한 목적식을 통해 의미적 규칙성을 반영

#### **(3) 제안된 방법론** :
##### $ \hspace{0.15cm} \cdot{} $ 단어 쌍의 로그 co-occurrence를 임베딩 내적과 bias의 합으로 근사함
##### $ \hspace{0.15cm} \cdot{} $ weighted least squares 목적함수를 사용해 빈도 불균형을 완화함
##### $ \hspace{0.15cm} \cdot{} $ 학습된 target과 context 벡터의 합을 최종 단어 임베딩으로 구성함

<b></b>

> ## **방법론(Method)**

#### **(0) co-occurrence ratio** :
##### $ \hspace{0.15cm} \cdot{} $ 코퍼스의 동시출현 행렬 $ X \in{} \mathbb{R}^{V \times V} $에서 각 원소 $ X_{ij} $는 단어 $ i $ 주변 윈도우 내에서 단어 $ j $가 등장한 횟수를 의미함
##### $ \hspace{0.45cm} \Rightarrow{} X_{ij} = \sum_{t} \mathbb{I}\{ w_{t}=i,\, w_{t+j}=j \} \;\; \text{ where } \, \mathbb{I}\{\cdot\}: \text{indicator function within window} $
##### $ \hspace{0.15cm} \cdot{} $ 조건부 확률을 $ P_{ij}=P(j|i)=X_{ij}/X_{i} $로 정의함
##### $ \hspace{0.15cm} \cdot{} $ 단어 의미의 차이는 probe 단어 $ k $에 대한 조건부 확률의 비율 $ P(k|i)/P(k|j) $의 로그로 표현할 수 있음
##### $ \hspace{0.15cm} \Rightarrow{} (\mathbf{w}_{i}-\mathbf{w}_{j})^{\top}\tilde{\mathbf{w}}_{k} = \log{}{\frac{P_{ik}}{P_{jk}}} \;\; \text{ where } \, P_{ik}=X_{ik}/X_{i}, \;\; P_{jk}=X_{jk}/X_{j}, \;\; \mathbf{w}_{i},\tilde{\mathbf{w}}_{k} \in{} \mathbb{R}^{d} $

#### **(1) GloVe(Global Vectors for Word Representation)** :
##### $ \hspace{0.15cm} \cdot{} $ 단어 $ i $가 주어졌을 때 단어 $ j $가 등장할 확률 $ P(j|i) $를 근사하도록 임베딩을 학습하는 것을 목표
##### $ \hspace{0.15cm} \Rightarrow{} \displaystyle{} \theta^{*} = \arg\max_{\theta{}} \sum_{i=1}^{V}\sum_{j=1}^{V} \log p_{\theta{}}(j|i) $
##### $ \hspace{0.45cm} \text{where } \, V : \text{vocabulary size}, \;\; \theta{} = \{ \mathbf{w}_{i}, \tilde{\mathbf{w}}_{j}, b_{i}, \tilde{b}_{j} \} $
##### $ \hspace{0.15cm} \cdot{} $ 조건부 확률 $ p_{\theta{}}(j|i) $는 단어 벡터 내적과 bias를 이용한 softmax 형태로 정의됨
##### $ \hspace{0.15cm} \Rightarrow{} p_{\theta{}}(j|i) = \displaystyle{} \frac{\exp{}\big(\mathbf{w}_{i}^{\top}\tilde{\mathbf{w}}_{j} + b_{i} + \tilde{b}_{j}\big)}{\displaystyle{} \sum_{j'\in{}\mathcal{V}} \exp{}\big(\mathbf{w}_{i}^{\top}\tilde{\mathbf{w}}_{j'} + b_{i} + \tilde{b}_{j'}\big)} $
##### $ \hspace{0.45cm} \text{where } \, \mathbf{w}_{i}, \tilde{\mathbf{w}}_{j} \in{} \mathbb{R}^{d} : \text{target, context embeddings}, \;\; b_{i}, \tilde{b}_{j} \in{} \mathbb{R} : \text{bias terms} $
##### $ \hspace{0.15cm} \cdot{} $ 실제 코퍼스의 조건부 확률 $ P(j|i)=X_{ij}/X_{i} $를 모델 확률 $ p_{\theta{}}(j|i) $로 근사하도록 학습하며, 두 확률의 로그 비율 관계는 다음과 같음
##### $ \hspace{0.15cm} \Rightarrow{} (\mathbf{w}_{i}-\mathbf{w}_{j})^{\top}\tilde{\mathbf{w}}_{k} = \log{}{\frac{P(k|i)}{P(k|j)}} \;\; \text{ where } \, P(k|i)=X_{ik}/X_{i}, \;\; X_{i}=\sum_{m}X_{im} $
##### $ \hspace{0.15cm} \cdot{} $ 이를 모든 단어쌍 $(i,j)$에 대해 근사하기 위해, 로그 동시출현 빈도를 내적과 bias의 합으로 표현하는 로그-선형 회귀식을 사용함
##### $ \hspace{0.15cm} \Rightarrow{} \log{}{X_{ij}} \approx \mathbf{w}_{i}^{\top}\tilde{\mathbf{w}}_{j} + b_{i} + \tilde{b}_{j} $
##### $ \hspace{0.15cm} \cdot{} $ 학습은 관측된 $ X_{ij} $에 대해 weighted least squares 손실을 최소화하도록 수행됨
##### $ \hspace{0.15cm} \Rightarrow{} J^{*} = \arg\min_{\theta{}} \sum_{i=1}^{V}\sum_{j=1}^{V} f(X_{ij})\big(\mathbf{w}_{i}^{\top}\tilde{\mathbf{w}}_{j} + b_{i} + \tilde{b}_{j} - \log{}{X_{ij}}\big)^{2} $
##### $ \hspace{0.45cm} \text{where } \, f(x)=(x/x_{\max})^{\alpha} \text{ if } x<x_{\max} \text{ otherwise } 1, \;\; x_{\max}=100, \;\; \alpha=3/4 $
##### $ \hspace{0.15cm} \cdot{} $ 최종 단어 임베딩은 target 벡터와 context 벡터의 합으로 구성함
##### $ \hspace{0.15cm} \Rightarrow{} \mathbf{v}_{i} = \mathbf{w}_{i} + \tilde{\mathbf{w}}_{i} $
##### $ \hspace{0.45cm} \text{where } \, \mathbf{v}_{i} \in{} \mathbb{R}^{d} : \text{final embedding for word } i $

<b></b>

> ## **실험(Experiments)**

#### **(1) 실험 설정** :
##### $ \hspace{0.15cm} \cdot{} $ 데이터셋
| Item | Description |
|------|--------------|
| Training dataset | Wikipedia 2014 + Gigaword 5 (6B tokens) / Common Crawl (42B tokens) |
| Vocabulary size($V$) | 400K (default), up to 1.9M |
| token size($T$) | 6B / 42B |
| Evaluation dataset | Google analogy, MSR analogy, WordSim353, MEN |
##### $ \hspace{0.15cm} \cdot{} $ 모델
| Item | Description |
|------|--------------|
| Model type | Global log-bilinear regression (GloVe) |
| Architecture | target, context embeddings + bias |
| Parameter size | $\approx 2Vd + 2V$ |
| Hyper parameter | $d=50/100/200/300$, window size = 10, $x_{\max}=100$, $\alpha=0.75$ |
##### $ \hspace{0.15cm} \cdot{} $ 훈련 및 평가
| Item | Description |
|------|--------------|
| Loss function | Weighted least squares on $\log X_{ij}$ |
| Evaluation metric | Analogy accuracy, Spearman correlation |
| Baseline models | Skip-gram, CBOW, LSA (SVD-based) |
| Training hardware | CPU multi-threaded |
| Training time | ? |

#### **(2) 실험 결과** :
##### $ \hspace{0.15cm} \cdot{} $ Word analogy에서 semantic·syntactic 관계 모두에서 높은 정확도 달성 ($71.7\%$ at 6B, $75.0\%$ at 42B)
##### $ \hspace{0.15cm} \cdot{} $ Word similarity에서 Spearman 상관 WS-353(75.9), MC(83.6), RG(82.9)로 강한 상관성을 보임
##### $ \hspace{0.15cm} \cdot{} $ NER(CoNLL-2003) 실험에서 CRF pipeline의 F1 향상($88.3\%$)을 달성함
##### $ \hspace{0.15cm} \cdot{} $ 임베딩 차원이 증가함에 따라 성능은 향상되나 약 200D 이후 수익 체감이 발생함
##### $ \hspace{0.15cm} \cdot{} $ 작은 윈도우는 syntactic 관계에, 큰 윈도우는 semantic 관계에 유리함
##### $ \hspace{0.15cm} \cdot{} $ 대규모 코퍼스일수록 전역 의미 정보가 강화되어 semantic accuracy가 상승함
##### $ \hspace{0.15cm} \cdot{} $ Skip-gram, CBOW 대비 시간–정확도 효율성이 높음

<b></b>

> ## **결론(Conclusion)**

#### **(1)** GloVe는 전역 co-occurrence ratio를 내적 선형식으로 모델링하여 의미 관계를 공간적으로 표현함

#### **(2)** weighted least squares 목적함수와 weighting function으로 고빈도·저빈도 단어의 영향을 균형화함

#### **(3)** count-based와 predict-based 접근의 장점을 결합하여 대규모 코퍼스에서도 효율성과 성능을 모두 달성함

<b></b>

> ## **부록(Appendix)**

<b></b>

> ## **비평(Commentary)**

<b></b>

> ## **참고 문헌(Reference)**

#### **(1) 관련 후속 논문** :
##### $ \hspace{0.15cm} \cdot{} $ **[~]**
##### $ \hspace{0.15cm} \cdot{} $ **[~]**
##### $ \hspace{0.15cm} \cdot{} $ **[~]**

#### **(2) 기타 참고 자료** :
##### $ \hspace{0.15cm} \cdot{} $ **[~]**
##### $ \hspace{0.15cm} \cdot{} $ **[~]**
##### $ \hspace{0.15cm} \cdot{} $ **[~]**